In [3]:
#import dependencies 
import pandas as pd 
from sqlalchemy import create_engine
import psycopg2
import config 

In [ ]:
#read in data into dataframe 
beer_url = pass
breweries_url = pass 

#beers_df = pd.read_csv(beer_url)
#brewery_df = pd.read_csv(brewery_df)
#beer_df.head(2)
#brewery_df.head(2)

In [10]:
beer_df = pd.read_csv('../../beer_cleaned.csv')
breweries_df = pd.read_csv('../../brewery_cleaned.csv')

beer_df.head(2)
breweries_df.head(2)

,abv,ibu,beer_id,beer_name,style,brewery_id,ounces
0,0.061,60.0,1979,Bitter Bitch,American Pale Ale (APA),177,12.0
1,0.099,92.0,1036,Lower De Boom,American Barleywine,368,8.4


,brewery_id,brewery_name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY


In [12]:
#psycopg2 connection to aws through postgres 
conn = psycopg2.connect(database='postgres', user=config.user, password=config.db_password, host=config.host)
cur = conn.cursor()

cur.execute('''CREATE TABLE beers ( 
                    abv DOUBLE PRECISION, 
                    ibu DOUBLE PRECISION, 
                    beer_id TEXT, 
                    beer_name TEXT, 
                    style TEXT, 
                    brewery_id TEXT PRIMARY KEY, 
                    ounces DOUBLE PRECISION );
                CREATE TABLE breweries (
                    brewery_id TEXT PRIMARY KEY, 
                    brewery_name TEXT, 
                    city TEXT, 
                    state TEXT );''')

conn.commit()
conn.close()

In [13]:
#sqlalcehmy database connection
database = f'postgres://{config.user}:{config.db_password}@{config.host}/postgres'
engine = create_engine(database)

In [14]:
# load beers dataframe 
beer_df.to_sql(name='beers', if_exists='append', index=False, con=engine)

In [15]:
#load breweries dataframe 
breweries_df.to_sql(name='breweries', if_exists='append', index=False, con=engine)

In [17]:
# querying beers table 
pd.read_sql_query('''SELECT * 
                  FROM beers
                  LIMIT 3;''' ,con=engine)

,abv,ibu,beer_id,beer_name,style,brewery_id,ounces
0,0.061,60.0,1979,Bitter Bitch,American Pale Ale (APA),177,12.0
1,0.099,92.0,1036,Lower De Boom,American Barleywine,368,8.4
2,0.079,45.0,1024,Fireside Chat,Winter Warmer,368,12.0


In [18]:
#querying breweries table 
pd.read_sql_query('''SELECT *
                  FROM breweries
                  LIMIT 3;''' ,con=engine)

,brewery_id,brewery_name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA


In [21]:
#joing tables 
pd.read_sql_query('''SELECT COUNT(*)
                    FROM beers as be
                    LEFT JOIN breweries as br
                    ON (be.brewery_id = br.brewery_id)
                    ;''',con=engine)


,count
0,1326


In [28]:
#joing tables 
pd.read_sql_query('''SELECT *
                    FROM beers as be
                    LEFT JOIN breweries as br
                    ON (be.brewery_id = br.brewery_id
                    LIMIT 10 
                    ;''',con=engine)

,abv,ibu,beer_id,beer_name,style,brewery_id,ounces,brewery_id,brewery_name,city,state
0,0.061,60.0,1979,Bitter Bitch,American Pale Ale (APA),177,12.0,177,18th Street Brewery,Gary,IN
1,0.099,92.0,1036,Lower De Boom,American Barleywine,368,8.4,368,21st Amendment Brewery,San Francisco,CA
2,0.079,45.0,1024,Fireside Chat,Winter Warmer,368,12.0,368,21st Amendment Brewery,San Francisco,CA
3,0.044,42.0,876,Bitter American,American Pale Ale (APA),368,12.0,368,21st Amendment Brewery,San Francisco,CA
4,0.049,17.0,802,Hell or High Watermelon Wheat (2009),Fruit / Vegetable Beer,368,12.0,368,21st Amendment Brewery,San Francisco,CA
5,0.049,17.0,800,21st Amendment Watermelon Wheat Beer (2006),Fruit / Vegetable Beer,368,12.0,368,21st Amendment Brewery,San Francisco,CA
6,0.070,70.0,799,21st Amendment IPA (2006),American IPA,368,12.0,368,21st Amendment Brewery,San Francisco,CA
7,0.070,70.0,797,Brew Free! or Die IPA (2008),American IPA,368,12.0,368,21st Amendment Brewery,San Francisco,CA
8,0.070,70.0,796,Brew Free! or Die IPA (2009),American IPA,368,12.0,368,21st Amendment Brewery,San Francisco,CA
9,0.085,52.0,531,Special Edition: Allies Win The War!,English Strong Ale,368,12.0,368,21st Amendment Brewery,San Francisco,CA
